In [4]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
import sklearn.metrics as skm
import torch
import cv2
import yaml
import albumentations as A
from ultralytics.data.build import YOLODataset
import ultralytics.data.build as build

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False

fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:

class YOLOWeightedDataset(YOLODataset):
    def __init__(self, *args, mode="train", **kwargs):
        """
        Initialize the WeightedDataset.

        Args:
            class_weights (list or numpy array): A list or array of weights corresponding to each class.
        """

        super(YOLOWeightedDataset, self).__init__(*args, **kwargs)

        self.train_mode = "train" in self.prefix

        # You can also specify weights manually instead
        self.count_instances()
        class_weights = np.sum(self.counts) / self.counts

        # Aggregation function
        self.agg_func = np.mean

        self.class_weights = np.array(class_weights)
        self.weights = self.calculate_weights()
        self.probabilities = self.calculate_probabilities()

    def count_instances(self):
        """
        Count the number of instances per class

        Returns:
            dict: A dict containing the counts for each class.
        """
        self.counts = [0 for i in range(len(self.data["names"]))]
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)
            for id in cls:
                self.counts[id] += 1

        self.counts = np.array(self.counts)
        self.counts = np.where(self.counts == 0, 1, self.counts)

    def calculate_weights(self):
        """
        Calculate the aggregated weight for each label based on class weights.

        Returns:
            list: A list of aggregated weights corresponding to each label.
        """
        weights = []
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)

            # Give a default weight to background class
            if cls.size == 0:
              weights.append(1)
              continue

            # Take mean of weights
            # You can change this weight aggregation function to aggregate weights differently
            weight = self.agg_func(self.class_weights[cls])
            weights.append(weight)
        return weights

    def calculate_probabilities(self):
        """
        Calculate and store the sampling probabilities based on the weights.

        Returns:
            list: A list of sampling probabilities corresponding to each label.
        """
        total_weight = sum(self.weights)
        probabilities = [w / total_weight for w in self.weights]
        return probabilities

    def __getitem__(self, index):
        """
        Return transformed label information based on the sampled index.
        """
        # Don't use for validation
        if not self.train_mode:
            return self.transforms(self.get_image_and_label(index))
        else:
            index = np.random.choice(len(self.labels), p=self.probabilities)
            return self.transforms(self.get_image_and_label(index))

build.YOLODataset = YOLOWeightedDataset

In [ ]:
from ultralytics import YOLO

IMGSZ = 1024
run_dir = f'train_{FOLD}'
print(run_dir)
!rm -rf runs/detect/{run_dir}

MODEL_NAME = 'yolo11s'
model = YOLO(f'{MODEL_NAME}.pt')
results = model.train(data='datasets/config.yaml',
                      epochs=50,
                      imgsz=IMGSZ,
                      name=run_dir,
                      device=0,
                      batch=20,
                      optimizer='AdamW',
                      lr0=3e-4,
                      momentum=0.9,
                      weight_decay=1e-2,
                      close_mosaic=30,
                      seed=RANDOM_STATE)

train_0


100%|██████████| 18.4M/18.4M [00:00<00:00, 176MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=datasets/config.yaml, epochs=50, time=None, patience=100, batch=20, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train_0, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=41, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=30, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 17.1MB/s]


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytic

...

                   all        979       7637      0.743      0.757      0.785       0.48

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      13.4G      1.117     0.7345      1.095         85       1024: 100%|██████████| 460/460 [06:20<00:00,  1.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:35<00:00,  1.42s/it]


                   all        979       7637      0.752      0.756      0.786      0.481

50 epochs completed in 5.878 hours.
Optimizer stripped from runs/detect/train_0/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train_0/weights/best.pt, 19.2MB

Validating runs/detect/train_0/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,421,701 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:36<00:00,  1.47s/it]


                   all        979       7637      0.751      0.756      0.786      0.481
Corn_Cercospora_Leaf_Spot        171       1326      0.635      0.464      0.549      0.234
      Corn_Common_Rust         60        315      0.746      0.524      0.648      0.376
          Corn_Healthy         49        302      0.698      0.567      0.678      0.377
Corn_Northern_Leaf_Blight         20         57      0.792      0.935      0.951       0.68
           Corn_Streak         71        636      0.695      0.714      0.775      0.452
 Pepper_Bacterial_Spot        120        445       0.65      0.497      0.549      0.192
     Pepper_Cercospora         37         91      0.751      0.764      0.786      0.339
   Pepper_Early_Blight          2         15      0.942          1      0.995      0.943
       Pepper_Fusarium         40         84      0.894      0.964      0.981      0.566
        Pepper_Healthy         49        130      0.816      0.885      0.905      0.522
    Pepper_Late